In [2]:
%load_ext autoreload
%autoreload 2
from brancharchitect.io import  parse_newick
from brancharchitect.plot.tree_plot import plot_circular_trees_in_a_row
from brancharchitect.leaforder.old.tree_order_optimisation_local import optimize_unique_splits, optimize_s_edge_splits
from brancharchitect.leaforder.circular_distances import circular_distance_tree_pair
from brancharchitect.leaforder.tree_order_optimisation_classic import (
    improve_single_pair_classic,
)

def test_improve_single_pair_on_example_one():
    # Example trees provided
    tree_newick = "(((B,C),((E,F),(D,A))),(O1,O2));(((B,C),A),(D,(E,F)),(O1,O2));"
    # Parse the trees
    tree1, tree2 = parse_newick(
        tree_newick, order=["B", "C", "A", "E", "F", "D", "O1", "O2"]
    )

    # Store original orders
    original_order_tree1 = tree1.get_current_order()
    original_order_tree2 = tree2.get_current_order()

    # Calculate distance before optimization
    distance_before = circular_distance_tree_pair(tree2, tree1)

    # Test improve_single_pair with s-edge and unique splits optimization
    plot_circular_trees_in_a_row([tree1, tree2])

    improved = improve_single_pair_classic(
        tree1, tree2, [optimize_s_edge_splits, optimize_unique_splits]
    )
    
    improved = improve_single_pair_classic(
        tree2, tree1, [optimize_s_edge_splits, optimize_unique_splits]
    )

    plot_circular_trees_in_a_row([tree1, tree2])
    
    distance_after = circular_distance_tree_pair(tree1, tree2)
    new_order_tree2 = tree2.get_current_order()

    # Check results
    if distance_after < distance_before:
        print("Optimization successful: Circular distance reduced.")
    elif distance_after == distance_before:
        print("Circular distance unchanged after optimization.")
    else:
        print("Optimization unsuccessful: Circular distance increased.")

    print("Original Taxa Order Tree1:", original_order_tree1)
    print("Original Taxa Order Tree2:", original_order_tree2)
    print(f"Circular distance before optimization: {distance_before}")
    print(f"Circular distance after optimization: {distance_after}")
    print("New Taxa Order Tree2 after optimization:", new_order_tree2)


def test_improve_single_pair_on_example_two():
    # Example trees
    tree_newick = "(((B,C),((E,F),(D,A))),(O1,O2));((((B,C),A),(D,(E,F))),(O1,O2));"
    # Parse the trees
    tree1, tree2 = parse_newick(
        tree_newick, order=["B", "C", "A", "E", "F", "D", "O1", "O2"]
    )

    original_order_tree1 = tree1.get_current_order()
    original_order_tree2 = tree2.get_current_order()

    distance_before = circular_distance_tree_pair(tree2, tree1)

    # Test improve_single_pair with s-edge and unique splits optimization
    plot_circular_trees_in_a_row([tree1, tree2])
    improved = improve_single_pair_classic(
        tree1, tree2, [optimize_s_edge_splits, optimize_unique_splits]
    )
    improved = improve_single_pair_classic(
        tree2, tree1, [optimize_s_edge_splits, optimize_unique_splits]
    )
    
    plot_circular_trees_in_a_row([tree1, tree2])
    distance_after = circular_distance_tree_pair(tree1, tree2)
    new_order_tree2 = tree2.get_current_order()

    if distance_after < distance_before:
        print("Optimization successful: Circular distance reduced.")
    elif distance_after == distance_before:
        print("Circular distance unchanged after optimization.")
    else:
        print("Optimization unsuccessful: Circular distance increased.")

    print("Original Taxa Order Tree1:", original_order_tree1)
    print("Original Taxa Order Tree2:", original_order_tree2)
    print(f"Circular distance before optimization: {distance_before}")
    print(f"Circular distance after optimization: {distance_after}")
    print("New Taxa Order Tree2 after optimization:", new_order_tree2)

    assert (
        distance_after < distance_before
    ), "Distance should reduce after optimization."


def test_improve_single_pair_reverts_on_no_improvement():
    """
    Test that when improve_single_pair does not find any improvement,
    the rotation functions revert the tree to its original state.
    """
    tree_newick = "(A,(B,C));(A,(B,C));"
    tree1, tree2 = parse_newick(tree_newick, order=["A", "B", "C"])

    initial_distance = circular_distance_tree_pair(tree1, tree2)

    original_tree2_order = tree2.get_current_order()
    plot_circular_trees_in_a_row([tree1, tree2])

    improved = improve_single_pair_classic(
        tree1, tree2, [optimize_s_edge_splits, optimize_unique_splits]
    )

    improved = improve_single_pair_classic(
        tree2, tree1, [optimize_s_edge_splits, optimize_unique_splits]
    )

    plot_circular_trees_in_a_row([tree1, tree2])

    final_distance = circular_distance_tree_pair(tree1, tree2)

    # No improvement expected
    assert not improved, "No improvement should be found."
    assert final_distance == initial_distance, "Distance should remain the same."
    assert (
        tree2.get_current_order() == original_tree2_order
    ), "Tree should revert to original."

    print("Test passed: Rotations correctly revert when no improvement is observed.")


def test_optimize_s_edge_and_unique_splits_on_common_splits_example_one():
    # Tests optimize_s_edge_splits and optimize_unique_splits independently on a complex example
    tree_newick = (
        "(((B,C),((E,F),(D,A))),(O1,O2));"
        + "((((B,C),A),(D,(E,F))),(O1,O2));"
        + "(((B,D),(C,E)),(O1,F),(A,O2));"
    )
    tree1, tree2, tree3 = parse_newick(
        tree_newick, order=["A", "B", "C", "D", "E", "F", "O1", "O2"]
    )

    plot_circular_trees_in_a_row([tree1, tree2])
    distance_before_1_2 = circular_distance_tree_pair(tree1, tree2)

    # Apply s-edge and unique splits optimization individually (if desired) or in combination
    improve_single_pair_classic(
        tree1, tree2, [optimize_s_edge_splits, optimize_unique_splits]
    )

    plot_circular_trees_in_a_row([tree1, tree2])
    distance_after_1_2 = circular_distance_tree_pair(tree1, tree2)

    print(f"Circular distance before optimization 1-2: {distance_before_1_2}")
    print(f"Circular distance after optimization 1-2: {distance_after_1_2}")


def test_improve_single_pair_with_common_splits_example_two():
    tree_newick = "(((B,C),((E,F),(D,A))),(O1,O2));((((B,C),A),(D,(E,F))),(O1,O2));(((B,D),(C,E)),(O1,F),(A,O2));"
    tree1, tree2, tree3 = parse_newick(
        tree_newick, order=["A", "B", "C", "D", "E", "F", "O1", "O2"]
    )

    distance_before_1_2 = circular_distance_tree_pair(tree1, tree2)

    plot_circular_trees_in_a_row([tree1, tree2])

    improved  = improve_single_pair_classic(
        tree1=tree1,
        tree2=tree2,
        rotation_functions=[optimize_s_edge_splits, optimize_unique_splits],
    )

    improved  = improve_single_pair_classic(
        tree1=tree2,
        tree2=tree1,
        rotation_functions=[optimize_s_edge_splits, optimize_unique_splits],
    )

    plot_circular_trees_in_a_row([tree1, tree2])

    distance_after_1_2 = circular_distance_tree_pair(tree1, tree2)

    print(f"Circular distance before optimization 1-2: {distance_before_1_2}")
    print(f"Circular distance after optimization 1-2: {distance_after_1_2}")


def test_improve_single_pair_with_common_splits_example_three():
    tree_newick = (
        "(((B,C),((E,F),(D,A))),(O1,O2));"
        "(((B,C),((E,F),(D,A))),(O1,O2));"
        "(((B,D),(C,E)),(O1,F),(A,O2));"
    )
    tree1, tree2, tree3 = parse_newick(
        tree_newick, order=["A", "B", "C", "D", "E", "F", "O1", "O2"]
    )

    plot_circular_trees_in_a_row([tree1, tree2, tree3])

    distance_before = circular_distance_tree_pair(tree1, tree2)

    improved  = improve_single_pair_classic(
        tree1, tree2, [optimize_s_edge_splits, optimize_s_edge_splits]
    )

    plot_circular_trees_in_a_row([tree1, tree2, tree3])

    distance_after = circular_distance_tree_pair(tree1, tree2)

    print(f"Circular distance before optimization 1-2: {distance_before}")
    print(f"Circular distance after optimization 1-2: {distance_after}")


def test_improve_single_pair_repeated_on_common_splits_example_four():
    tree_newick = "(((B,C),((E,F),(D,A))),(O1,O2));" "(((B,D),(C,E)),(O1,F),(A,O2));"
    tree1, tree2 = parse_newick(
        tree_newick, order=["A", "B", "C", "D", "E", "F", "O1", "O2"]
    )

    distance_before = circular_distance_tree_pair(tree1, tree2)
    plot_circular_trees_in_a_row([tree1, tree2])

    for i in range(10):

        improved  = improve_single_pair_classic(
            tree1, tree2, [optimize_unique_splits, optimize_s_edge_splits]
        )
        improved  = improve_single_pair_classic(
            tree2, tree1, [optimize_unique_splits, optimize_s_edge_splits]
        )

    plot_circular_trees_in_a_row([tree1, tree2])
    distance_after = circular_distance_tree_pair(tree1, tree2)

    print(f"Circular distance before optimization 1-2: {distance_before}")
    print(f"Circular distance after optimization 1-2: {distance_after}")


def test_optimize_s_edge_and_unique_splits_on_common_splits_example_five():
    tree_newick = "(((B,C),((E,F),(D,A))),(O1,O2));" "(((B,D),(C,E)),(O1,F),(A,O2));"
    tree1, tree2 = parse_newick(
        tree_newick, order=["A", "B", "C", "D", "E", "F", "O1", "O2"]
    )

    distance_before = circular_distance_tree_pair(tree1, tree2)
    plot_circular_trees_in_a_row([tree1, tree2])

    improved  = improve_single_pair_classic(
        tree1, tree2, [optimize_s_edge_splits, optimize_s_edge_splits]
    )

    improved  = improve_single_pair_classic(tree2, tree1, [optimize_s_edge_splits, optimize_s_edge_splits])

    distance_after = circular_distance_tree_pair(tree1, tree2)
    plot_circular_trees_in_a_row([tree1, tree2])
    
    print(f"Circular distance before optimization 1-2: {distance_before}")
    print(f"Circular distance after optimization 1-2: {distance_after}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from IPython.display import display, HTML
import xml.etree.ElementTree as ET

def add_background_to_svg_string(svg_string: str, background_color: str = "#f0f0f0") -> str:
    """Parses an SVG string, adds a background rectangle, and returns the modified SVG string."""
    try:
        # ET.fromstring can be sensitive to XML declarations, remove if present
        if svg_string.startswith("<?xml"):
            svg_string = svg_string.split("?>", 1)[-1].strip()
        
        svg_root = ET.fromstring(svg_string)
        
        # Ensure it's an SVG element
        if svg_root.tag.endswith("svg"): # Handles potential namespace prefixes
            background_rect = ET.Element("rect")
            background_rect.set("width", "100%")
            background_rect.set("height", "100%")
            background_rect.set("fill", background_color)
            svg_root.insert(0, background_rect) # Insert as the first child
            
            # ET.tostring returns bytes, so decode to unicode
            return ET.tostring(svg_root, encoding="unicode")
        else:
            # Not an SVG root, return original
            return svg_string
            
    except ET.ParseError as e:
        print(f"Error parsing SVG: {e}")
        return svg_string # Return original on error

def test_improve_single_pair_on_example_one():
    # Example trees provided
    tree_newick = "(((B,C),((E,F),(D,A))),(O1,O2));(((B,C),A),(D,(E,F)),(O1,O2));"
    # Parse the trees
    tree1, tree2 = parse_newick(
        tree_newick, order=["B", "C", "A", "E", "F", "D", "O1", "O2"]
    )

    # Store original orders
    original_order_tree1 = tree1.get_current_order()
    original_order_tree2 = tree2.get_current_order()

    # Calculate distance before optimization
    distance_before = circular_distance_tree_pair(tree2, tree1)

    # Test improve_single_pair with s-edge and unique splits optimization
    print("Trees before optimization:")
    svg_string_before = plot_circular_trees_in_a_row([tree1, tree2])
    svg_with_bg_before = add_background_to_svg_string(svg_string_before, "lightgrey") # Add background
    display(HTML(svg_with_bg_before))

    improved = improve_single_pair_classic(
        tree1, tree2, [optimize_s_edge_splits, optimize_unique_splits]
    )
    
    improved = improve_single_pair_classic(
        tree2, tree1, [optimize_s_edge_splits, optimize_unique_splits]
    )

    print("\\nTrees after optimization:")
    svg_string_after = plot_circular_trees_in_a_row([tree1, tree2])
    svg_with_bg_after = add_background_to_svg_string(svg_string_after, "lightcyan") # Add different background
    display(HTML(svg_with_bg_after))
    
    distance_after = circular_distance_tree_pair(tree1, tree2)
    new_order_tree2 = tree2.get_current_order()

    # Check results
    if distance_after < distance_before:
        print("Optimization successful: Circular distance reduced.")
    elif distance_after == distance_before:
        print("Circular distance unchanged after optimization.")
    else:
        print("Optimization unsuccessful: Circular distance increased.")

    print("Original Taxa Order Tree1:", original_order_tree1)
    print("Original Taxa Order Tree2:", original_order_tree2)
    print(f"Circular distance before optimization: {distance_before}")
    print(f"Circular distance after optimization: {distance_after}")
    print("New Taxa Order Tree2 after optimization:", new_order_tree2)


tree_newick_row = "(((B,C),((E,F),(D,A))),(O1,O2));((((B,C),A),(D,(E,F))),(O1,O2));"

tree_list = parse_newick(
    tree_newick_row, order=["B", "C", "A", "E", "F", "D", "O1", "O2"]
) # parse_newick returns a list of Node objects

svg_output_row = plot_circular_trees_in_a_row(
    roots=tree_list, 
    size=250,  # Optional: adjust size
    margin=20   # Optional: adjust margin
)

display(HTML(svg_output_row))
svg_with_bg_after = add_background_to_svg_string(svg_output_row, "lightcyan") # Add different background

NameError: name 'parse_newick' is not defined